In [26]:
import numpy as np
from scipy.optimize import minimize


In [27]:
X = np.random.rand(150,75)
(N,D) = X.shape

In [28]:
theta_1 = np.empty(5)
theta_1.fill(10)

theta_2 = np.empty(5)
theta_2.fill(-10)

theta = np.hstack((theta_1,theta_2))
np.random.shuffle(theta)

theta


array([-10., -10.,  10.,  10., -10., -10.,  10.,  10.,  10., -10.])

In [29]:
theta_rest = np.zeros(65)
theta = np.hstack((theta,theta_rest))
theta

array([-10., -10.,  10.,  10., -10., -10.,  10.,  10.,  10., -10.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.])

In [30]:
epsilon = 0.1 * np.random.randn(150) + 0
epsilon

array([ 0.2075461 , -0.09220067,  0.02944101, -0.02253925, -0.0269738 ,
       -0.22736057,  0.04841124, -0.06310904,  0.08871596, -0.04487165,
       -0.06776207,  0.02199168,  0.07216606, -0.09827645,  0.10346962,
       -0.00669326, -0.06281561,  0.00627887,  0.18550423, -0.09133291,
       -0.02411795,  0.07454208, -0.07918575,  0.04483258, -0.08422056,
        0.02729786,  0.00213366,  0.06713065,  0.11409969, -0.10718391,
        0.08796095, -0.01780226, -0.12025829, -0.03919039,  0.02063028,
       -0.15733327,  0.03227476, -0.10952325, -0.00561213, -0.03011879,
       -0.14564812, -0.02444326,  0.05289373,  0.03660689,  0.09273325,
       -0.04183642,  0.00314062,  0.01111406, -0.15519623,  0.0582409 ,
        0.06035278, -0.16870049, -0.0751043 ,  0.18296295, -0.07658836,
       -0.26635603, -0.09070951,  0.07076634,  0.01314771,  0.08456275,
        0.07520031,  0.04371737, -0.21841282,  0.07759048,  0.15224305,
       -0.00861925, -0.05025865,  0.00393656,  0.06201864,  0.18

In [31]:
y = np.dot(X,theta) + epsilon
y

array([  5.39783937e+00,   3.25199324e+00,   1.07647649e+01,
         1.41704338e+01,   1.81551663e+01,  -1.16578454e+01,
        -1.47295523e+01,  -2.94201121e+00,  -4.90207594e-01,
        -7.65052967e+00,  -2.06140327e+00,   8.46452738e+00,
         1.56148357e+01,   4.64053185e+00,   1.63825873e+01,
         7.40533011e+00,  -9.83610383e+00,   3.35979816e+00,
         3.71101974e+00,  -9.30323805e+00,  -5.01946603e+00,
        -8.67172713e+00,  -1.53179278e+01,   2.10670736e+00,
         8.90857472e+00,  -7.86274197e+00,   9.55167578e+00,
         1.37091453e+01,  -5.59256457e+00,   4.82386282e+00,
        -8.97019919e+00,  -4.88017257e+00,  -5.93381650e+00,
         5.93634512e+00,   1.20372762e+01,  -7.47072622e+00,
        -4.11422722e+00,   2.35680820e+00,  -9.58266610e+00,
        -1.06405760e+01,   9.30689338e+00,   4.25462247e+00,
         1.19840596e+01,   1.18302210e+00,   1.42813229e+01,
         4.68665609e-01,  -1.63732946e+01,   4.58940092e+00,
        -1.03900529e+01,

In [32]:
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=70, random_state=42)

In [33]:
X_test,X_validation,y_test,y_validation = train_test_split(X_test, y_test, test_size=50, random_state=42)

In [45]:
from scipy.optimize import minimize
lambda_loss = np.zeros((11,2))
lambda_theta = np.zeros((11,D))
theta_learned = np.zeros(D)

def ridge(Lambda):
  def ridge_obj(theta):
    return ((np.linalg.norm(np.dot(X_train,theta) - y_train))**2)/(2*N) + Lambda*(np.linalg.norm(theta))**2
  return ridge_obj

def compute_loss(theta):
  return ((np.linalg.norm(np.dot(X_validation,theta) - y_validation))**2)/(2*N)

for i in range(-5,6):
  Lambda = 10**i;
  theta_opt = minimize(ridge(Lambda), theta)
  lambda_loss[i] = Lambda, compute_loss(theta_opt.x) 
  lambda_theta[i] = theta_opt.x

least_loss_lambda = lambda_loss[np.argmin(lambda_loss,axis=0)[1],[0]]
print 'least loss' , np.amin(lambda_loss,axis=0)[1]
theta_learned = lambda_theta[np.argmin(lambda_loss,axis=0)[1]]
#get the theta for the least_loss_lambda


least loss 0.0346274685553


### Observation 

* Minimized Loss = 0.0346274685553

In [35]:
theta_learned

array([ -9.68364556e+00,  -9.74328975e+00,   1.01723193e+01,
         9.81888657e+00,  -1.01778599e+01,  -1.00372018e+01,
         9.62177538e+00,   9.90326523e+00,   9.86871564e+00,
        -1.00640145e+01,  -6.35615516e-02,   4.62569737e-02,
         7.42820225e-03,   9.23064006e-02,   5.72009826e-02,
         1.08012481e-01,  -2.91259826e-02,   4.50652354e-02,
         2.20915981e-02,  -9.62685892e-02,  -8.85664865e-02,
        -1.33595567e-01,   3.94454504e-02,   1.32375672e-01,
        -1.43469902e-01,   1.31730975e-01,  -3.52186169e-01,
        -1.87796943e-01,   5.31247307e-03,   5.21746550e-02,
         2.48934331e-01,   1.38275778e-01,  -2.51638008e-02,
         1.19008733e-01,  -1.87700395e-01,  -1.41764420e-01,
        -2.20798929e-01,  -9.81381949e-03,  -4.85775106e-01,
        -1.17668059e-01,   3.79510460e-01,   1.38533009e-01,
         9.77280832e-02,   8.14904769e-02,   3.06155098e-02,
         9.85940577e-02,   1.78583791e-01,   8.06489979e-03,
        -1.85666260e-01,

### Observation

* All values of the learned theta are non-zero. The true vector has first 10 values as non-zero


In [41]:
theta_learned[theta_learned < 10**-3] = 0
print(theta_learned.shape)
theta_learned

(75,)


array([  0.00000000e+00,   0.00000000e+00,   1.01723193e+01,
         9.81888657e+00,   0.00000000e+00,   0.00000000e+00,
         9.62177538e+00,   9.90326523e+00,   9.86871564e+00,
         0.00000000e+00,   0.00000000e+00,   4.62569737e-02,
         7.42820225e-03,   9.23064006e-02,   5.72009826e-02,
         1.08012481e-01,   0.00000000e+00,   4.50652354e-02,
         2.20915981e-02,   0.00000000e+00,   0.00000000e+00,
         0.00000000e+00,   3.94454504e-02,   1.32375672e-01,
         0.00000000e+00,   1.31730975e-01,   0.00000000e+00,
         0.00000000e+00,   5.31247307e-03,   5.21746550e-02,
         2.48934331e-01,   1.38275778e-01,   0.00000000e+00,
         1.19008733e-01,   0.00000000e+00,   0.00000000e+00,
         0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         0.00000000e+00,   3.79510460e-01,   1.38533009e-01,
         9.77280832e-02,   8.14904769e-02,   3.06155098e-02,
         9.85940577e-02,   1.78583791e-01,   8.06489979e-03,
         0.00000000e+00,

### Observation

* Results after scaling the values to be > 10**-3

* Number of components with true value non-zero estimated to be zero = 5 out of 10
* Number of components with true value zero estimated to be non-zero = 

In [16]:
from sklearn.linear_model import Ridge
clf = Ridge(alpha=1.0)
clf.fit(X_train, y_train) 
y_obs = clf.predict(X_validation)

np.sum(y_obs - y_validation)

0.34391457674983172